<a href="https://colab.research.google.com/github/lorransr/ice_fire_and_data/blob/master/Uma_can%C3%A7%C3%A3o_de_Gelo%2C_Fogo_e_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Uma canção de Fogo, Gelo e Dados

Game of Thrones acabou. O seriado épico da HBO que deriva da série literária de George RR Martin que começou a ser escrita em 1996, as crônicas de Gelo e Fogo, chegou ao seu fim no dia 19 de maio de 2019. A série, foi trazida a TV por David Benioff e D.B Weiss em 17 de abril de 2011. Devido a segmentação do conteúdo, cada vez mais nichado, provavelmente não veremos outra série evento como esta.  

GoT é grandiosa em roteiro, personagens, ambientação mas principalmente em escala. A série foi rodada em 10 países diferentes. A irlanda sozinha, teve 50 locações. Segundo a Forbes foram emitidos 19722 documentos de viagem, 68143 quartos de hotel foram alugados e 243 filmagens foram agendadas durante a série. 
No que diz respeito as primeiras 7 temporadas, GoT recebeu 132 nomeações ao Emmy, e ganhou 42. Foram 7 nomeações ao globo de ouro e 1 vitória; 18 nomeações ao SAG awards (7 vitorias), 17 Nomeações ao Critics' Choice Award (1 Vitória) e 7 vitórias no AFI awards.

Ao todo, Game of Thrones é exibida em 207 paises, e possui transmissão simultanea em 194 deles. Alias,é muito comum as séries perderem audiência conforme se passam os anos. Porém, game of thrones apresentou uma constante subida. O show foi de uma média de 9,3 milhões de espectadores nos EUA, na primeira temporada, para a invejável audiência de 32,8 milhões na sétima.

O seriado prendeu a atenção de todos nesses ultimos 9 anos. Especialmente de Jeffrey Lancaster, que compilou o [mais incrivel conjunto de dados](https://medium.com/@jeffrey.lancaster/the-ultimate-game-of-thrones-dataset-a100c0cf35fb), sob Game of Thrones. Na verdade, Jeffrey cruzou varias bases disponiveis na internet, e chegou ao ponto de rever todos episodios de todas as temporadas e anotou diversas informações cena por cena, afinal, por que não?

Com os dados compilados por ele, eu pretendo responder algumas perguntas a respeito do seriado como: Quem personagem apareceu mais vezes? Quem ficou mais tempo em tela? Personagens femininos tiveram a mesma relevância na trama que personagens masculinos? Afinal, tem tanto sexo assim em Game of Thrones? Quanta gente morreu? 



In [0]:
#importando as libs
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from pandas.io.json import json_normalize
import json

In [2]:
!git clone -l -s git://github.com/lorransr/ice_fire_and_data.git cloned-repo
%cd cloned-repo
!ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 35 (delta 16), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (35/35), 935.45 KiB | 4.87 MiB/s, done.
Resolving deltas: 100% (16/16), done.
/content/cloned-repo
'character_sex .csv'   README.md
 episodes.json	       Uma_canção_de_Gelo,_Fogo_e_Dados.ipynb


In [0]:
with open('../cloned-repo/episodes.json') as f:
    d = json.load(f)

recapitulando: 
* Puxei os dados do Json "episodes"; 
* Segmentei por cenas e personagens;
* mantive os campos meta referentes ao episódio;
* mantive o inicio e o fim da cena;

In [4]:

#https://www.kaggle.com/jboysen/quick-tutorial-flatten-nested-json-in-pandas
#trabalhando com o json
raw = json_normalize(data = d['episodes'],
                     record_path = ["scenes","characters"],
                     meta=["seasonNum","episodeNum",["scenes","sceneStart"],
                           ["scenes","sceneEnd"],"episodeAirDate"])
raw.head(10)

,alive,born,killedBy,mannerOfDeath,married,name,sex,title,weapon,seasonNum,episodeNum,scenes.sceneStart,scenes.sceneEnd,episodeAirDate
0,NaN,NaN,NaN,NaN,NaN,Gared,NaN,NaN,NaN,1,1,0:00:40,0:01:45,2011-04-17
1,NaN,NaN,NaN,NaN,NaN,Waymar Royce,NaN,NaN,NaN,1,1,0:00:40,0:01:45,2011-04-17
2,NaN,NaN,NaN,NaN,NaN,Will,NaN,NaN,NaN,1,1,0:00:40,0:01:45,2011-04-17
3,NaN,NaN,NaN,NaN,NaN,Gared,NaN,NaN,NaN,1,1,0:01:45,0:03:24,2011-04-17
4,NaN,NaN,NaN,NaN,NaN,Waymar Royce,NaN,NaN,NaN,1,1,0:01:45,0:03:24,2011-04-17
5,NaN,NaN,NaN,NaN,NaN,Will,NaN,NaN,NaN,1,1,0:01:45,0:03:24,2011-04-17
6,NaN,NaN,NaN,NaN,NaN,Will,NaN,NaN,NaN,1,1,0:03:24,0:03:31,2011-04-17
7,False,NaN,NaN,NaN,NaN,Wight Wildling Girl,NaN,NaN,NaN,1,1,0:03:24,0:03:31,2011-04-17
8,NaN,NaN,NaN,NaN,NaN,Will,NaN,NaN,NaN,1,1,0:03:31,0:03:38,2011-04-17
9,NaN,NaN,NaN,NaN,NaN,Gared,NaN,NaN,NaN,1,1,0:03:44,0:05:36,2011-04-17


In [0]:
df = raw

In [6]:
df.columns

Index(['alive', 'born', 'killedBy', 'mannerOfDeath', 'married', 'name', 'sex',
       'title', 'weapon', 'seasonNum', 'episodeNum', 'scenes.sceneStart',
       'scenes.sceneEnd', 'episodeAirDate'],
      dtype='object')

In [7]:
df.columns=['alive', 'born', 'killedBy', 'mannerOfDeath', 'married', 'name', 'sex',
            'title', 'weapon', 'seasonNum', 'episodeNum', 'sceneStart',
            'sceneEnd', 'episodeAirDate']
df.dtypes

alive             object
born              object
killedBy          object
mannerOfDeath     object
married           object
name              object
sex               object
title             object
weapon            object
seasonNum          int64
episodeNum         int64
sceneStart        object
sceneEnd          object
episodeAirDate    object
dtype: object

In [0]:
df["sceneEnd_s"]=df["sceneEnd"].str.split(":")
df["sceneEnd_s"]=df["sceneEnd_s"].str[0].astype(int)*3600+df["sceneEnd_s"].str[1].astype(int)*60+df["sceneEnd_s"].str[2].astype(int)

df["sceneStart_s"]=df["sceneStart"].str.split(":")
df["sceneStart_s"]=df["sceneStart_s"].str[0].astype(int)*3600+df["sceneStart_s"].str[1].astype(int)*60+df["sceneStart_s"].str[2].astype(int)

df["screenTime"] = df["sceneEnd_s"]-df["sceneStart_s"]


In [9]:
df.head()

,alive,born,killedBy,mannerOfDeath,married,name,sex,title,weapon,seasonNum,episodeNum,sceneStart,sceneEnd,episodeAirDate,sceneEnd_s,sceneStart_s,screenTime
0,NaN,NaN,NaN,NaN,NaN,Gared,NaN,NaN,NaN,1,1,0:00:40,0:01:45,2011-04-17,105,40,65
1,NaN,NaN,NaN,NaN,NaN,Waymar Royce,NaN,NaN,NaN,1,1,0:00:40,0:01:45,2011-04-17,105,40,65
2,NaN,NaN,NaN,NaN,NaN,Will,NaN,NaN,NaN,1,1,0:00:40,0:01:45,2011-04-17,105,40,65
3,NaN,NaN,NaN,NaN,NaN,Gared,NaN,NaN,NaN,1,1,0:01:45,0:03:24,2011-04-17,204,105,99
4,NaN,NaN,NaN,NaN,NaN,Waymar Royce,NaN,NaN,NaN,1,1,0:01:45,0:03:24,2011-04-17,204,105,99


## Quem apareceu mais vezes em cena 

In [10]:
df["name2"] = df["name"]

rank = df.groupby(by = ["seasonNum","name"])[["name2"]].count().reset_index()
rank.columns = ["seasonNum","name","count"]
rank.sort_values(by=["seasonNum","count"],ascending=False,inplace=True)

rank.head()

,seasonNum,name,count
1018,8,Daenerys Targaryen,171
1034,8,Jon Snow,137
1058,8,Tyrion Lannister,126
1021,8,Drogon,98
1033,8,Jaime Lannister,82


In [0]:
#main characters - todos que apareceram mais que cinco vezes
rank["main"]=rank["count"].apply(lambda x: 0 if x<10 else 1)

In [12]:
import altair as alt


bars = alt.Chart(rank).mark_bar().encode(
        y= alt.Y('name',sort=alt.EncodingSortField("count",order="descending")),
        x='count',
        color=alt.Color('seasonNum:O',scale=alt.Scale(scheme ="blueorange" )),
        order = alt.Order("seasonNum",sort="ascending")
                ).configure_axis(grid=False
                ).configure_view(strokeWidth=0)

bars

Chart({
  config: Config({
    axis: AxisConfig({
      grid: False
    }),
    view: ViewConfig({
      strokeWidth: 0
    })
  }),
  data:       seasonNum                  name  count  main
  1018          8    Daenerys Targaryen    171     1
  1034          8              Jon Snow    137     1
  1058          8      Tyrion Lannister    126     1
  1021          8                Drogon     98     1
  1033          8       Jaime Lannister     82     1
  1010          8            Arya Stark     81     1
  1031          8             Grey Worm     75     1
  1019          8        Davos Seaworth     73     1
  1037          8            Lord Varys     65     1
  1052          8        Sandor Clegane     60     1
  1053          8           Sansa Stark     58     1
  1014          8      Brienne of Tarth     57     1
  1013          8            Bran Stark     54     1
  1051          8         Samwell Tarly     50     1
  1057          8    Tormund Giantsbane     49     1
  1035          8         Jorah Mormont     47     1
  1016          8      Cersei Lannister     43     1
  1043          8             Missandei     39     1
  1046          8         Podrick Payne     36     1
  1049          8               Rhaegal     36     1
  1027          8                Gendry     35     1
  1030          8        Gregor Clegane     34     1
  1025          8         Euron Greyjoy     29     1
  1012          8      Beric Dondarrion     27     1
  1048          8                Qyburn     27     1
  1038          8        Lyanna Mormont     25     1
  1042          8            Melisandre     22     1
  1055          8        The Night King     22     1
  1056          8         Theon Greyjoy     21     1
  1067          8            Yohn Royce     20     1
  ...         ...                   ...    ...   ...
  121           1   Wight Wildling Girl      2     0
  0             1        Addam Marbrand      1     0
  2             1                Armeca      1     0
  6             1      Beric Dondarrion      1     0
  10            1      Catspaw Assassin      1     0
  15            1        Dothraki Crone      1     0
  27            1               Hot Pie      1     0
  32            1         Jafer Flowers      1     0
  35            1         Jaremy Rykker      1     0
  39            1             Jon Arryn      1     0
  41            1         Jonos Bracken      1     0
  44            1                  Joss      1     0
  45            1      Joyeuse Erenford      1     0
  48            1  King's Landing Baker      1     0
  51            1   Lannister Messenger      1     0
  53            1           Leo Lefford      1     0
  54            1           Little Bird      1     0
  55            1      Lommy Greenhands      1     0
  64            1               Mhaegen      1     0
  79            1    Red Keep Stableboy      1     0
  83            1      Rickard Karstark      1     0
  91            1          Ryger Rivers      1     0
  99            1          Stevron Frey      1     0
  100           1                  Stiv      1     0
  101           1         Street Urchin      1     0
  107           1            Tobho Mott      1     0
  108           1                Tomard      1     0
  113           1                 Varly      1     0
  114           1           Vayon Poole      1     0
  118           1                Wallen      1     0
  
  [1068 rows x 4 columns],
  encoding: FacetedEncoding({
    color: Color({
      scale: Scale({
        scheme: 'blueorange'
      }),
      shorthand: 'seasonNum:O'
    }),
    order: Order({
      shorthand: 'seasonNum',
      sort: 'ascending'
    }),
    x: X({
      shorthand: 'count'
    }),
    y: Y({
      shorthand: 'name',
      sort: EncodingSortField({
        field: 'count',
        order: 'descending'
      })
    })
  }),
  mark: 'bar'
})

In [13]:
#Quantidade de Personagens
rank.name.nunique()

572

O grafico, ficou um pouco grande para o artigo. Afinal, foram 572 personagens, contanto papeis menores, por isso optei pegar apenas os 10 que aparecem mais para construir a visualização

In [0]:
#criando a lista com os 10 personagens que aparecem mais
selected_names = rank.groupby("name")[["count"]].sum(
                                               ).reset_index(
                                               ).sort_values(by = "count",
                                                             ascending = False
                                                            ).head(15).name

In [0]:
#criando o DF com apenas os personagens que aparecem mais
rank_15 = rank[rank.name.isin(selected_names)]

In [16]:
#plotando o gráfico
bars = alt.Chart(rank_15).mark_bar().encode(
        y= alt.Y('name',sort=alt.EncodingSortField("count",order="descending"),title=""),
        x=alt.X('count', title = ""),
        color=alt.Color('seasonNum:O',scale=alt.Scale(scheme ="blueorange" )),
        order = alt.Order("seasonNum",sort="ascending")
                ).configure_axis(grid=False
                ).configure_view(strokeWidth=0)

bars.properties(
    width=1000,
    height=600,
    title = "Quantidade de vezes em cena",

)

Chart({
  config: Config({
    axis: AxisConfig({
      grid: False
    }),
    view: ViewConfig({
      strokeWidth: 0
    })
  }),
  data:       seasonNum                name  count  main
  1018          8  Daenerys Targaryen    171     1
  1034          8            Jon Snow    137     1
  1058          8    Tyrion Lannister    126     1
  1021          8              Drogon     98     1
  1033          8     Jaime Lannister     82     1
  1010          8          Arya Stark     81     1
  1019          8      Davos Seaworth     73     1
  1037          8          Lord Varys     65     1
  1052          8      Sandor Clegane     60     1
  1053          8         Sansa Stark     58     1
  1013          8          Bran Stark     54     1
  1051          8       Samwell Tarly     50     1
  1057          8  Tormund Giantsbane     49     1
  1035          8       Jorah Mormont     47     1
  1016          8    Cersei Lannister     43     1
  964           7            Jon Snow    105     1
  946           7  Daenerys Targaryen     80     1
  1001          7    Tyrion Lannister     71     1
  963           7     Jaime Lannister     70     1
  965           7       Jorah Mormont     65     1
  947           7      Davos Seaworth     53     1
  993           7      Sandor Clegane     51     1
  998           7  Tormund Giantsbane     47     1
  994           7         Sansa Stark     40     1
  949           7              Drogon     39     1
  970           7          Lord Varys     36     1
  937           7          Arya Stark     30     1
  945           7    Cersei Lannister     30     1
  942           7          Bran Stark     18     1
  992           7       Samwell Tarly     18     1
  ...         ...                 ...    ...   ...
  282           3      Davos Seaworth     15     1
  322           3          Lord Varys     14     1
  285           3              Drogon     10     1
  252           2    Tyrion Lannister     82     1
  141           2    Cersei Lannister     46     1
  231           2         Sansa Stark     43     1
  129           2          Arya Stark     42     1
  144           2  Daenerys Targaryen     38     1
  230           2      Sandor Clegane     30     1
  178           2            Jon Snow     28     1
  179           2       Jorah Mormont     22     1
  147           2      Davos Seaworth     21     1
  134           2          Bran Stark     19     1
  191           2          Lord Varys     15     1
  229           2       Samwell Tarly     14     1
  172           2     Jaime Lannister     10     1
  151           2              Drogon      8     0
  40            1            Jon Snow     44     1
  110           1    Tyrion Lannister     38     1
  13            1  Daenerys Targaryen     36     1
  3             1          Arya Stark     32     1
  7             1          Bran Stark     29     1
  11            1    Cersei Lannister     22     1
  42            1       Jorah Mormont     22     1
  94            1         Sansa Stark     22     1
  57            1          Lord Varys     18     1
  92            1       Samwell Tarly     16     1
  33            1     Jaime Lannister     15     1
  93            1      Sandor Clegane     14     1
  16            1              Drogon      9     0
  
  [115 rows x 4 columns],
  encoding: FacetedEncoding({
    color: Color({
      scale: Scale({
        scheme: 'blueorange'
      }),
      shorthand: 'seasonNum:O'
    }),
    order: Order({
      shorthand: 'seasonNum',
      sort: 'ascending'
    }),
    x: X({
      shorthand: 'count',
      title: ''
    }),
    y: Y({
      shorthand: 'name',
      sort: EncodingSortField({
        field: 'count',
        order: 'descending'
      }),
      title: ''
    })
  }),
  height: 600,
  mark: 'bar',
  title: 'Quantidade de vezes em cena',
  width: 1000
})

In [17]:
rank_15[rank_15["name"]=="Jon Snow"].describe()

,seasonNum,count,main
count,8.00000,8.000000,8.0
mean,4.50000,72.375000,1.0
std,2.44949,36.558906,0.0
min,1.00000,28.000000,1.0
25%,2.75000,42.250000,1.0
50%,4.50000,73.500000,1.0
75%,6.25000,87.000000,1.0
max,8.00000,137.000000,1.0


In [18]:
rank_15[rank_15["name"]=="Jon Snow"]["count"].sum()

579

Jon Snow é o campeão de vezes em cena, com uma média de 68 cenas por temporada. Chegando a marca de 579 cenas! É interessante observar que conforme a trama do seriado se aproxima do fim, a relevância de Jon toma cada vez mais peso na série. Personagens como Cercei e Tyrion possuem temporadas onde possuem muitas cenas.
Destaque para Drogon, o maior dragão de Daenerys, que esteve muito presente na oitava temporada, tanto que permitiu sua permanência na lista.

Agora, fica a questão, quem será que possui mais tempo de tela?

## Quem possui mais tempo em cena?

In [0]:
#criando o df com o tempo de tela
rank_time = df.groupby(["name","seasonNum","episodeNum"])[["screenTime"]].sum()
rank_time.reset_index(inplace=True)
rank_time.sort_values(by ="screenTime", ascending=False,inplace=True)

In [0]:
#tranformar o tempo de tela em minutos
rank_time["screenTime"] = rank_time["screenTime"]/60

In [21]:
#selecionando top 15 personagens
rank_time = rank_time[rank_time["name"].isin(selected_names)]
#top 5 por episodio
rank_time.head()

,name,seasonNum,episodeNum,screenTime
2975,Tyrion Lannister,8,4,39.600000
1330,Jon Snow,7,6,37.083333
2971,Tyrion Lannister,7,7,31.466667
1331,Jon Snow,7,7,31.433333
1384,Jorah Mormont,7,6,31.083333


In [22]:
#top 5 por temporada
rank_time.groupby("name")[["screenTime"]].sum().reset_index().sort_values("screenTime",ascending=False).head()

,name,screenTime
14,Tyrion Lannister,643.333333
7,Jon Snow,630.650000
3,Daenerys Targaryen,510.700000
2,Cersei Lannister,423.200000
12,Sansa Stark,411.783333


In [23]:
#heatmap do tempo de cena dos personagens
from altair.expr import datum
base = alt.Chart(rank_time).mark_rect().encode(
                                               x='episodeNum:O',
                                               y='seasonNum:O',
                                               column = "name:N",
                                               color=alt.Color('sum(screenTime):Q',
                                               scale=alt.Scale(scheme ="goldred" ))
                                              ).properties(
                                                           width=400,
                                                           height=400)
base
#base.facet(
          #row = "name:N"
          #)


Chart({
  data:                     name  seasonNum  episodeNum  screenTime
  2975    Tyrion Lannister          8           4   39.600000
  1330            Jon Snow          7           6   37.083333
  2971    Tyrion Lannister          7           7   31.466667
  1331            Jon Snow          7           7   31.433333
  1384       Jorah Mormont          7           6   31.083333
  2880  Tormund Giantsbane          7           6   30.466667
  566   Daenerys Targaryen          8           4   30.083333
  2488      Sandor Clegane          7           6   29.983333
  1323            Jon Snow          6           9   29.800000
  1604          Lord Varys          8           4   26.783333
  463     Cersei Lannister          7           7   26.333333
  1314            Jon Snow          5           8   24.150000
  611       Davos Seaworth          7           7   24.100000
  1185     Jaime Lannister          7           7   24.083333
  2940    Tyrion Lannister          4           2   23.883333
  1385       Jorah Mormont          7           7   23.700000
  1600          Lord Varys          7           7   23.433333
  2434       Samwell Tarly          4           9   23.133333
  1187     Jaime Lannister          8           2   23.100000
  1335            Jon Snow          8           4   22.883333
  2967    Tyrion Lannister          7           3   21.800000
  2973    Tyrion Lannister          8           2   21.716667
  434     Cersei Lannister          4           2   20.966667
  1306            Jon Snow          4           9   20.950000
  2876  Tormund Giantsbane          6           9   20.816667
  562   Daenerys Targaryen          7           7   20.433333
  1327            Jon Snow          7           3   19.883333
  2869  Tormund Giantsbane          5           8   19.816667
  1189     Jaime Lannister          8           4   19.583333
  1332            Jon Snow          8           1   19.566667
  ...                  ...        ...         ...         ...
  697               Drogon          7           7    1.200000
  2468      Sandor Clegane          2           7    1.166667
  682               Drogon          4           1    1.166667
  596       Davos Seaworth          5          10    1.150000
  1380       Jorah Mormont          7           1    1.100000
  683               Drogon          4           6    1.066667
  2859  Tormund Giantsbane          3           7    1.050000
  684               Drogon          5           2    1.050000
  2435       Samwell Tarly          4          10    1.033333
  257           Bran Stark          2           7    1.016667
  439     Cersei Lannister          4           7    1.000000
  2862  Tormund Giantsbane          4           3    0.950000
  258           Bran Stark          2           8    0.933333
  610       Davos Seaworth          7           6    0.883333
  111           Arya Stark          6          10    0.883333
  2470      Sandor Clegane          3           2    0.866667
  692               Drogon          7           1    0.833333
  678               Drogon          3           1    0.833333
  671               Drogon          1           3    0.816667
  681               Drogon          3          10    0.750000
  693               Drogon          7           3    0.600000
  1594          Lord Varys          6          10    0.566667
  74            Arya Stark          2           1    0.483333
  685               Drogon          5           5    0.433333
  2471      Sandor Clegane          3           3    0.416667
  278           Bran Stark          7           5    0.400000
  676               Drogon          2           6    0.333333
  2474      Sandor Clegane          3           7    0.216667
  689               Drogon          6           8    0.200000
  553   Daenerys Targaryen          6           8    0.200000
  
  [746 rows x 4 columns],
  encoding: FacetedEncoding({
    color: Color({
      scale: Scale({
        scheme: 'goldred'
      }),
      s

Quando se trata de tempo em cena, Tyrion segue como o personagem que mais esteve presente no seriado. Chegando ao ponto de aparecer quase que em 40 minutos do episódio 4 da ultima temporada. Seguido por Jon, e Dany. A proposito a mãe dos dragões possui 20% a menos de tempo em cena do que Jon Snow. Game of Thrones tem tantos personagens, e tantas linhas diferentes de arco que literalmente, nenhum dos personagens com mais tempo de tela aparecem em todos os episódios.

## Personagens masculinos e femininos possuem tempo igual de tela?

In [24]:
#importando o csv que diferencia personagem por sexo
url = "https://raw.githubusercontent.com/lorransr/ice_fire_and_data/master/character_sex%20.csv"
dim_sex = pd.read_csv(url)
dim_sex.head()

,name,sex
0,Alliser Thorne,male
1,Baby Sam,male
2,Balon Greyjoy,male
3,Barristan Selmy,male
4,Benjen Stark,male


In [0]:
#juntando os df
df = pd.merge(df,
         dim_sex[["name","sex"]],on="name",how="left")

In [0]:
#preparando o df
df_sex = df.groupby(by=["sex_y","seasonNum","episodeNum"])[["screenTime"]].sum()
df_sex.reset_index(inplace=True)

In [57]:
teste = df[df.sex_y.isnull()].groupby(by=["name"])[["screenTime"]].sum()
teste.sort_values(by="screenTime",ascending = False).head(50)

,screenTime
name,
Qhono,3573
Meryn Trant,2789
The Night King,1623
Qotho,1587
White Walker,1557
Myranda,1270
Randyll Tarly,1157
Anguy,1152
Bowen Marsh,1137


In [36]:
#plotando o gráfico

base = alt.Chart(df_sex).properties(
    width=500,
    height = 500
)


color_scale = alt.Scale(domain=['male', 'female'],
                        range=['#930416', '#eab64d'])

left = base.transform_filter(
    alt.datum.sex_y == 'female'
).encode(
    y=alt.Y('seasonNum:O', axis=None),
    x=alt.X('sum(screenTime):Q',
            title='tempo de tela',
            scale=alt.Scale(domain=(0, 110000)),
            sort=alt.SortOrder('descending')),
    color=alt.Color('sex_y:N', scale=color_scale, legend=None)
).mark_bar().properties(title='Mulheres')


middle = base.encode(
    y=alt.Y('seasonNum:O', axis=None),
    text=alt.Text('seasonNum:Q'),
).mark_text().properties(width=20)


right = base.transform_filter(
    alt.datum.sex_y == 'male'
).encode(
    y=alt.Y('seasonNum:O', axis=None),
    x=alt.X('sum(screenTime):Q',
            title='tempo de tela',
           scale=alt.Scale(domain=(0, 110000))),
    color=alt.Color('sex_y:N', scale=color_scale, legend=None)
).mark_bar().properties(title='Homens')

alt.concat(left, middle, right, spacing=5).properties(title = "Disparidade de genero em Tempo de Tela(M)")

ConcatChart({
  concat: [Chart({
    data:       sex_y  seasonNum  episodeNum  screenTime
    0    female          1           1        5245
    1    female          1           2        4361
    2    female          1           3        2103
    3    female          1           4        2571
    4    female          1           5        1990
    5    female          1           6        4189
    6    female          1           7        2299
    7    female          1           8        3591
    8    female          1           9        4264
    9    female          1          10        3295
    10   female          2           1        2786
    11   female          2           2        2207
    12   female          2           3        2790
    13   female          2           4        2646
    14   female          2           5        2618
    15   female          2           6        3030
    16   female          2           7        2763
    17   female          2           8        2188
    18   female          2           9        2140
    19   female          2          10        3411
    20   female          3           1        2627
    21   female          3           2        3695
    22   female          3           3        2109
    23   female          3           4        2782
    24   female          3           5        3374
    25   female          3           6        3070
    26   female          3           7        2872
    27   female          3           8        4112
    28   female          3           9        3828
    29   female          3          10        3171
    ..      ...        ...         ...         ...
    114    male          5           3        6410
    115    male          5           4        3868
    116    male          5           5        6285
    117    male          5           6        4278
    118    male          5           7        5591
    119    male          5           8        6225
    120    male          5           9        6195
    121    male          5          10        4391
    122    male          6           1        4006
    123    male          6           2        6711
    124    male          6           3        6533
    125    male          6           4        5824
    126    male          6           5        5016
    127    male          6           6        4283
    128    male          6           7        4385
    129    male          6           8        5398
    130    male          6           9        8993
    131    male          6          10        7800
    132    male          7           1        6930
    133    male          7           2        6210
    134    male          7           3        6528
    135    male          7           4        4290
    136    male          7           5        7630
    137    male          7           6       12927
    138    male          7           7       18289
    139    male          8           1        6720
    140    male          8           2       10665
    141    male          8           3        7726
    142    male          8           4       14937
    143    male          8           5        5959
    
    [144 rows x 4 columns],
    encoding: FacetedEncoding({
      color: Color({
        legend: None,
        scale: Scale({
          domain: ['male', 'female'],
          range: ['#930416', '#eab64d']
        }),
        shorthand: 'sex_y:N'
      }),
      x: X({
        scale: Scale({
          domain: (0, 110000)
        }),
        shorthand: 'sum(screenTime):Q',
        sort: SortOrder('descending'),
        title: 'tempo de tela'
      }),
      y: Y({
        axis: None,
        shorthand: 'seasonNum:O'
      })
    }),
    height: 500,
    mark: 'bar',
    title: 'Mulheres',
    transform: [FilterTransform({
      filter: (datum.sex_y === 'female')
    })],
    width: 500
  }), Chart({
    data:       sex_y  seasonNum  episodeNum  screenTime
    0    female          1     

## Como os personagens morreram?

In [28]:
df.head()

,alive,born,killedBy,mannerOfDeath,married,name,sex_x,title,weapon,seasonNum,episodeNum,sceneStart,sceneEnd,episodeAirDate,sceneEnd_s,sceneStart_s,screenTime,name2,sex_y
0,NaN,NaN,NaN,NaN,NaN,Gared,NaN,NaN,NaN,1,1,0:00:40,0:01:45,2011-04-17,105,40,65,Gared,NaN
1,NaN,NaN,NaN,NaN,NaN,Waymar Royce,NaN,NaN,NaN,1,1,0:00:40,0:01:45,2011-04-17,105,40,65,Waymar Royce,NaN
2,NaN,NaN,NaN,NaN,NaN,Will,NaN,NaN,NaN,1,1,0:00:40,0:01:45,2011-04-17,105,40,65,Will,NaN
3,NaN,NaN,NaN,NaN,NaN,Gared,NaN,NaN,NaN,1,1,0:01:45,0:03:24,2011-04-17,204,105,99,Gared,NaN
4,NaN,NaN,NaN,NaN,NaN,Waymar Royce,NaN,NaN,NaN,1,1,0:01:45,0:03:24,2011-04-17,204,105,99,Waymar Royce,NaN


In [0]:
#creating death data frame
death = df[df.killedBy.notnull()]
death = pd.DataFrame(death.mannerOfDeath.value_counts())
death.reset_index(inplace=True)
death.columns=["mannerOfDeath","counts"]
death = death[death["mannerOfDeath"]!=""]

#sorting death values
death.sort_values("counts",inplace=True,ascending = False)

#Calculando percentagem do total
total = sum(death.counts)
death["percentTotal"]=death["counts"]/total

In [30]:

bars = alt.Chart(death).mark_bar().encode(
    x=alt.X("percentTotal:Q",
            axis=alt.Axis(format=".0%"),
            title = "Porcentagem do Total de Mortes"),
    y=alt.Y("mannerOfDeath:N",
            title = "",
             sort=alt.EncodingSortField("counts",
                                        order="descending"))
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='counts:Q'
)

(bars + text).properties(height=600,
                         width = 1000,
                         title = "222 maneiras de se morrer em Westeros").configure_mark(opacity = 0.9,color='#4c516d')


LayerChart({
  config: Config({
    mark: MarkConfig({
      color: '#4c516d',
      opacity: 0.9
    })
  }),
  height: 600,
  layer: [Chart({
    data:            mannerOfDeath  counts  percentTotal
    0             Chest stab      26      0.117117
    1         Multiple stabs      24      0.108108
    2           Throat slash      20      0.090090
    3           Decapitation      15      0.067568
    4                  Arrow      15      0.067568
    5                Burning      12      0.054054
    6              Back stab       9      0.040541
    7                 Poison       9      0.040541
    8                 Dragon       8      0.036036
    11               Hanging       7      0.031532
    10               Mauling       7      0.031532
    9               Wildfire       7      0.031532
    12            Head crush       6      0.027027
    13             Neck stab       5      0.022523
    14             Face stab       5      0.022523
    15             Neck snap       5      0.022523
    16               Falling       4      0.018018
    17                 Giant       3      0.013514
    25                Beaten       2      0.009009
    24       Dead Night King       2      0.009009
    23         Fire and fall       2      0.009009
    22                Flayed       2      0.009009
    21              Gut stab       2      0.009009
    19           Throat stab       2      0.009009
    18                  Safe       2      0.009009
    20               Choking       2      0.009009
    36           Molten gold       1      0.004505
    44                  Door       1      0.004505
    43             Head stab       1      0.004505
    42                  Stab       1      0.004505
    41        Wight children       1      0.004505
    40              Drowning       1      0.004505
    39                Gutted       1      0.004505
    38             Moon Door       1      0.004505
    37           Poison dart       1      0.004505
    34            Torn apart       1      0.004505
    35       Giant crossbows       1      0.004505
    33        Tongue removal       1      0.004505
    32               Crushed       1      0.004505
    31                 Horse       1      0.004505
    30  Dragonglass Eye Stab       1      0.004505
    28           Shadow baby       1      0.004505
    27           Suffocation       1      0.004505
    26              Eye stab       1      0.004505
    45                  Boar       1      0.004505,
    encoding: FacetedEncoding({
      x: X({
        axis: Axis({
          format: '.0%'
        }),
        shorthand: 'percentTotal:Q',
        title: 'Porcentagem do Total de Mortes'
      }),
      y: Y({
        shorthand: 'mannerOfDeath:N',
        sort: EncodingSortField({
          field: 'counts',
          order: 'descending'
        }),
        title: ''
      })
    }),
    mark: 'bar'
  }), Chart({
    data:            mannerOfDeath  counts  percentTotal
    0             Chest stab      26      0.117117
    1         Multiple stabs      24      0.108108
    2           Throat slash      20      0.090090
    3           Decapitation      15      0.067568
    4                  Arrow      15      0.067568
    5                Burning      12      0.054054
    6              Back stab       9      0.040541
    7                 Poison       9      0.040541
    8                 Dragon       8      0.036036
    11               Hanging       7      0.031532
    10               Mauling       7      0.031532
    9               Wildfire       7      0.031532
    12            Head crush       6      0.027027
    13             Neck stab       5      0.022523
    14             Face stab       5      0.022523
    15             Neck snap       5      0.022523
    16               Falling       4      0.018018
    17                 Giant       3      0.013514
    25                Beaten       2      0.009009
    24       Dead Night King       2      0.009009
    23    

Esse gráfico nos mostra que em Westeros, a chance de morrer esfaqueado de alguma forma é bem alta - quase 30%. Por outro lado, o fim pode chegar das mais absurdas formas como esfaqueamento das genitalias ou um demônio recêm parido. Lembrando que apenas foram contabilizados as mortes que ocorreram em cena e afetaram personagens que receberam algum destaque, por parte da trama.

## Afinal, Tem tanto sexo assim em GoT?

In [0]:
sex = df[df.sex_x.notnull()]

In [32]:
sex_p = sex['sex_x'].apply(pd.Series)
sex_p

,with,type,when
91,[Ros],paid,present
92,[Tyrion Lannister],paid,present
138,[Daenerys Targaryen],rape,present
139,[Khal Drogo],rape,present
151,[Cersei Lannister],love,present
152,[Jaime Lannister],love,present
188,[Daenerys Targaryen],rape,present
189,[Khal Drogo],rape,present
227,[Daenerys Targaryen],love,present
228,[Khal Drogo],love,present


In [0]:
sex = pd.merge(sex,sex_p,how="left",left_index = True,right_index = True)

In [0]:
sex = sex.drop_duplicates(subset= ["sceneEnd","sceneStart"])

In [35]:
base = alt.Chart(sex).mark_rect().encode(
                                         x='episodeNum:O',
                                         y='seasonNum:O',
                                         color=alt.Color('count(type):Q',
                                         scale=alt.Scale(scheme ="bluepurple" ))
                                              ).properties(title = "Quantidade de sexo por episódio da temporada",
                                                           width=600,
                                                           height=600)
base

Chart({
  data:       alive born  ...      type     when
  91      NaN  NaN  ...      paid  present
  138     NaN  NaN  ...      rape  present
  151     NaN  NaN  ...      love  present
  188     NaN  NaN  ...      rape  present
  227     NaN  NaN  ...      love  present
  349     NaN  NaN  ...      love     past
  385     NaN  NaN  ...      paid  present
  492     NaN  NaN  ...      paid  present
  537     NaN  NaN  ...      love  present
  634     NaN  NaN  ...      paid  present
  897     NaN  NaN  ...      paid  present
  908     NaN  NaN  ...      paid   future
  994     NaN  NaN  ...    incest     past
  1018    NaN  NaN  ...      paid   future
  1029    NaN  NaN  ...      paid     past
  1199    NaN  NaN  ...       fun  present
  1202    NaN  NaN  ...      paid   future
  1242    NaN  NaN  ...  ambition  present
  1295    NaN  NaN  ...      love  present
  1309    NaN  NaN  ...      paid     past
  1373    NaN  NaN  ...      paid  present
  1695    NaN  NaN  ...    served   future
  1901    NaN  NaN  ...      love  present
  1938    NaN  NaN  ...      love     past
  2292    NaN  NaN  ...   coerced     past
  2561    NaN  NaN  ...      love   future
  2716    NaN  NaN  ...       fun  present
  2917    NaN  NaN  ...      love  present
  2998    NaN  NaN  ...       fun  present
  3103    NaN  NaN  ...      love  present
  3166    NaN  NaN  ...   torture  present
  3270    NaN  NaN  ...     magic  present
  4368    NaN  NaN  ...      rape  present
  4413    NaN  NaN  ...      paid  present
  4690    NaN  NaN  ...   married   future
  4804    NaN  NaN  ...      love  present
  4807    NaN  NaN  ...      love  present
  4810    NaN  NaN  ...      love  present
  4813    NaN  NaN  ...      love  present
  4988    NaN  NaN  ...      love   future
  5365    NaN  NaN  ...      love  present
  5545    NaN  NaN  ...       fun     past
  5562    NaN  NaN  ...      love     past
  5791    NaN  NaN  ...  marriage  present
  6126    NaN  NaN  ...      love  present
  6304    NaN  NaN  ...      rape  present
  6370    NaN  NaN  ...      love  present
  6378    NaN  NaN  ...      love  present
  6677    NaN  NaN  ...      paid   future
  7383    NaN  NaN  ...       fun  present
  7944    NaN  NaN  ...      love   future
  8422    NaN  NaN  ...      paid     past
  8818    NaN  NaN  ...      love  present
  8972    NaN  NaN  ...      love  present
  9963    NaN  NaN  ...      love  present
  10153   NaN  NaN  ...      paid  present
  10169   NaN  NaN  ...       fun     past
  10468   NaN  NaN  ...      love  present
  11293   NaN  NaN  ...      love   future
  
  [59 rows x 22 columns],
  encoding: FacetedEncoding({
    color: Color({
      scale: Scale({
        scheme: 'bluepurple'
      }),
      shorthand: 'count(type):Q'
    }),
    x: X({
      shorthand: 'episodeNum:O'
    }),
    y: Y({
      shorthand: 'seasonNum:O'
    })
  }),
  height: 600,
  mark: 'rect',
  title: 'Quantidade de sexo por episódio da temporada',
  width: 600
})